In [ ]:
import numpy
#give a expected median for each variable in the system, the last three are GC outputs, minus as a method to sum~0.
equation_inputs=[55,110,4000,0.4,10,100,-20,-5,-15]
num_weights=9


In [ ]:
import pandas as pd
data=pd.read_csv("/content/npc 18 test for colab 6 for ga.csv")


In [ ]:
import numpy as np
sol_per_pop=9


pop_size=(sol_per_pop,num_weights)
newpopulation=data.sample(n=9)
new_population=newpopulation.values
#new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

In [ ]:
def cal_pop_fitness(equation_inputs, pop):
    #calculating the fitness value of each solution in the current population
    #the fitness function calculates the sum of products between each input and its corresponding weight
    fitness=numpy.sum(pop*equation_inputs)
    return fitness


In [ ]:
def select_mating_pool(pop,fitness, num_parents):
    #selecting best individuals in the current generation as parents 
    #a empty array to hold the number of parents
    parents= numpy.empty((num_parents,pop.shape[1]))
    for parent_num in range(num_parents):
        # getting the lowest solution( in which I hope the fitness as the sum to be as closest to zero as possible)
        min_fitness_idx=numpy.where(fitness==numpy.min(fitness))
        min_fitness_idx=min_fitness_idx[0][0]
        # retrieve the solution that corresponds to such fitness value
        parents[parent_num,:]=pop[min_fitness_idx,:]
        #not gonna choose such solution anymore
        fitness[min_fitness_idx]=99999999.00
    return parents

  

In [ ]:
#crossover
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [ ]:
def mutation(offspring_crossover, num_mutations=1):
    mutations_counter = numpy.uint8(offspring_crossover.shape[1] / num_mutations)
    # Mutation changes a number of genes as defined by the num_mutations argument. The changes are random.
    for idx in range(offspring_crossover.shape[0]):
        gene_idx = mutations_counter - 1
        for mutation_num in range(num_mutations):
            # The random value to be added to the gene.
            random_value = numpy.random.uniform(-1.0, 1.0, 1)
            offspring_crossover[idx, gene_idx] = offspring_crossover[idx, gene_idx] + random_value
            gene_idx = gene_idx + mutations_counter
    return offspring_crossover

In [ ]:
num_generations = 500
num_parents_mating=5
for generation in range(num_generations):
    print("Generation : ", generation)
    # Measing the fitness of each chromosome in the population.
    fitness =cal_pop_fitness(equation_inputs, new_population)

    # Selecting the best parents in the population for mating.
    parents =select_mating_pool(new_population, fitness, 
                                      num_parents_mating)

    # Generating next generation using crossover.
    offspring_crossover =crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], num_weights))

    # Adding some variations to the offsrping using mutation.
    offspring_mutation =mutation(offspring_crossover)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

    # The best result in the current iteration.
    print("Best result : ", numpy.min(numpy.sum(new_population*equation_inputs, axis=1)))

# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness =cal_pop_fitness(equation_inputs, new_population)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = numpy.where(fitness == numpy.min(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])